# `swift_too` module

## Swift_PlanQuery example - querying planned *Swift* observations

### API version = 1.2, `swifttools` version = 2.2

#### Author: Jamie A. Kennea (Penn State)

The Swift_PlanQuery class allows for querying the database of observations have have been planned for observation by Swift, otherwise known as the "Pre-Planned Science Timeline" (PPST). This allows the user to find out not only what was planned, but what will be observed by Swift in the future.

Two important caveats:

1. For past observations, what should have been down is mostly an academic exercise, as due to TOO uploads and GRBs, Swift's pre-planned observations can be obliterated. If you want to know what Swift *did* observe, do a Swift_ObsQuery

2. Swift's planning is only done days into the future, so if you want to know what is going to happen weeks from now, you're out of luck.

This therefore is mostly useful if you want to know what Swift is observing *today* and *tomorrow*.

In [1]:
from swift_too import Swift_PlanQuery
from datetime import datetime,timedelta

### Setting up our username and secret key

Set up the class, giving our username and shared secret. These can be set up [on the Swift TOO website](https://www.swift.psu.edu/toop/too.php). After you log in, you will find your shared secret, and can modify it as necessary under the [Update Account Info](https://www.swift.psu.edu/toop/account_info.php) link. 

In [2]:
username = 'myuser'
shared_secret = 'mysharedsecret'

However, if you don't have a user or just don't want to use it, you can submit this request anonymously. In this case you just give the username and shared secret as "anonymous"

In [3]:
username = 'anonymous'
shared_secret = 'anonymous'

Note: as of `swifttools 2.2`, anonymous login is the default, so if you do not give the username and shared_secret, it will log in as 'anonymous'

### First Query

First query example: What is Swift planned to observe now? Quite easy to construct. Here it is:

In [4]:
plan = Swift_PlanQuery(begin=datetime.utcnow())
plan

2021-12-15 21:35:00,2021-12-15 22:04:00,GJ 3631,00014941013,1740


So the above should have reported back a single target. When you just give `begin` with no `end`, it simply returns the observation which is taking place at the time given. So what is happening at this time tomorrow?

In [5]:
plan = Swift_PlanQuery(begin=datetime.utcnow()+timedelta(days=1))
plan

2021-12-16 21:34:00,2021-12-16 21:50:00,MKN 6,00035461079,960


Did the above query return anything? If not, then there simply isn't a plan for tomorrow. So how can we determine how far into the future Swift has a plan for? The `ppstmax` parameter. Let's check it out:

In [6]:
f"PPSTs available up to {plan.ppstmax}"

'PPSTs available up to 2021-12-17 00:00:00'

This gives the date of the last planned target. If the query for what Swift is observing this time tomorrow didn't return anything, this date is probably the end of the UT day today. If the time you're running this is before midday on a Monday-Thursday, this is as expected. Typically the plan for tomorrow is submitted for upload around 12pm Eastern time. The exception to this rule is for the weekend plan. Typically on Thursday at 12pm we submit a plan for Friday and Saturday, and on Friday at 12pm we submit a plan for Sunday-Monday. This allows us to cover the weekend, while working a 5 day week.

### Second query example

Seeing a whole day's PPST. So let's see the whole day's Plan that goes farthest into the future.

In [7]:
dayplan = Swift_PlanQuery(begin=plan.ppstmax-timedelta(days=1),end=plan.ppstmax)
dayplan

2021-12-15 23:44:00,2021-12-16 00:00:00,OGP_0982,03103494002,960
2021-12-16 00:00:00,2021-12-16 00:14:00,OGP_0100,03104415004,840
2021-12-16 00:14:00,2021-12-16 00:41:00,saa-cold-150-12,00074011070,1620
2021-12-16 00:41:00,2021-12-16 01:01:00,OJ287,00035905178,1200
2021-12-16 01:01:00,2021-12-16 01:26:00,CDF-N,00095931056,1500
2021-12-16 01:26:00,2021-12-16 01:39:00,OGP_0198,03104130005,780
2021-12-16 01:39:00,2021-12-16 01:55:00,OGP_1260,03103159001,960
2021-12-16 01:55:00,2021-12-16 02:21:00,saa-cold-350-01,00076048055,1560
2021-12-16 02:21:00,2021-12-16 02:26:00,OGP_5272,03104469001,300
2021-12-16 02:26:00,2021-12-16 02:52:00,eRASSt_J104938+234140,00014968001,1560
2021-12-16 02:52:00,2021-12-16 02:58:00,NGC 4725,03110926034,360


That's a lot of targets! Note that like Swift_ObsQuery it's showing you all the individual snapshots during the day. If you want to group these into observations, you can do that. Remember the `observations` method returns a dictionary, with obsid as the key, and the total observation details as the value. You can display it as a table in Jupyter Notebooks simply:

In [8]:
dayplan.observations

2021-12-15 23:44:00,2021-12-16 00:00:00,OGP_0982,03103494002,960,0
2021-12-16 00:00:00,2021-12-16 00:14:00,OGP_0100,03104415004,840,0
2021-12-16 00:14:00,2021-12-16 00:41:00,saa-cold-150-12,00074011070,1620,0
2021-12-16 00:41:00,2021-12-16 12:04:00,OJ287,00035905178,1860,0
2021-12-16 01:01:00,2021-12-16 01:26:00,CDF-N,00095931056,1500,0
2021-12-16 01:26:00,2021-12-16 01:39:00,OGP_0198,03104130005,780,0
2021-12-16 01:39:00,2021-12-16 20:48:00,OGP_1260,03103159001,1680,0
2021-12-16 01:55:00,2021-12-16 02:21:00,saa-cold-350-01,00076048055,1560,0
2021-12-16 02:21:00,2021-12-16 11:53:00,OGP_5272,03104469001,720,0
2021-12-16 02:26:00,2021-12-16 07:29:00,eRASSt_J104938+234140,00014968001,2160,0
2021-12-16 02:52:00,2021-12-16 02:58:00,NGC 4725,03110926034,360,0
